In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

from osgeo import gdal, osr
import pandas as pd
os.getcwd()

In [ ]:
#os.chdir("../")

In [ ]:
#ncols         9720
#nrows         9159
#xllcorner     -27.174992029555
#yllcorner     -37.791665508412
#cellsize      0.0083333333333333
#NODATA_value  -9999
# seperated by a space

from osgeo import gdal, osr

AEZ_raster_in=os.path.join("data", "raster","AEZ_2009","afr_aez09.asc")
AEZ_raster_csv=os.path.join("data", "raster","AEZ_2009","afr_aez09.csv")

AEZ_raster_out=os.path.join("data", "raster","AEZ_2009","afr_aez09.tif")

with open(AEZ_raster_in, 'r', encoding = 'ANSI') as fp:
    data = fp.read()




In [ ]:
asc_header_data=data[:153]
AEZ_raster_headers=os.path.join("data", "raster","AEZ_2009","afr_aez09_headers.asc")

g = open(AEZ_raster_headers,'w',encoding='ANSI')

# write the data (but not the first 153 characters, which are metadata)
g.write(asc_header_data)

# close the new csv file
g.close()


In [ ]:
all_asc_data=data[153:]
data=None

In [ ]:
g = open(AEZ_raster_csv,'w',encoding='ANSI')

# write the data (but not the first 153 characters, which are metadata)
g.write(all_asc_data)

# close the new csv file
g.close()

In [ ]:
AEZ_data_frame=pd.read_csv(AEZ_raster_csv,
                           low_memory=False, 
                           encoding='ANSI', 
                           header=None, 
                           sep=' ')

In [ ]:
AEZ_data_frame.shape # should be 9159 rows and 9720 columns

In [ ]:

all_data_types=AEZ_data_frame.dtypes
all_data_types.unique()

In [ ]:
AEZ_df_as_float = pd.DataFrame(np.zeros(AEZ_data_frame.shape), columns=None)

partial_failure=[]

for row in AEZ_data_frame.index:
    try:
        AEZ_df_as_float.iloc[row,:]=AEZ_data_frame.iloc[row,:].astype(float)
    except ValueError:
        for column in AEZ_data_frame.columns:
            try:
                AEZ_df_as_float.iloc[row,column]=float(AEZ_data_frame.iloc[row,column])
                if row in partial_failure:
                    continue
                else:
                    partial_failure.append(row)
            except ValueError:
                 AEZ_df_as_float.iloc[row,column] = np.nan

#print(partial_failure) use this print statement to see the rows which contin only NAs
    
    

# there are some rows with data in them which is not numerical and looks like weird symbols when you open the asc file as plaintext
# I plotted the data to see where these are and whether they explain why the data is longer than expected
# to do that they need to be recorded as missing values rather than weird values



In [ ]:

# creating an array of every nth point point
number_of_rows=AEZ_df_as_float.shape[0]
number_of_columns=AEZ_df_as_float.shape[1]

number_of_data_points_to_skip = 10 # take every 10th point

total_row_intervals = int(np.divide(number_of_rows,number_of_data_points_to_skip)) #rounds down the division to the nearest integer
total_column_intervals = int(np.divide(number_of_columns, number_of_data_points_to_skip)) #rounds down the division to the nearest integer

total_number_of_data_points=total_row_intervals*total_column_intervals
rows_of_array = np.zeros(total_number_of_data_points)
columns_of_array = np.zeros(total_number_of_data_points)
aez_value = np.zeros(total_number_of_data_points)

for skipped_column in range(total_column_intervals): 
    for skipped_row in range(total_row_intervals):
        data_index = skipped_row + total_row_intervals*skipped_column
                
        columns_of_array[data_index]= AEZ_df_as_float.columns[skipped_column*number_of_data_points_to_skip]
        rows_of_array[data_index] = AEZ_df_as_float.index[skipped_row*number_of_data_points_to_skip]
        aez_value[data_index] = AEZ_df_as_float.iloc[skipped_row*number_of_data_points_to_skip,skipped_column*number_of_data_points_to_skip]

In [ ]:
# plotting as array
fig, ax= plt.subplots()
plt.scatter(x=columns_of_array,y=rows_of_array.max()-rows_of_array,c=aez_value,marker='s', s=.2)
plt.show()

In [ ]:
# plotting using imshow

#AEZ_array=AEZ_df_as_float.to_numpy()
#fig,ax=plt.subplots(figsize=(19.4,18.6))
#plt.imshow(AEZ_array, aspect="auto") 
#plt.show()

In [ ]:
africa_aez_temp = AEZ_df_as_float.dropna(axis=0, how='all') #dropping rows with 'all' missing values
africa_aez_temp = africa_aez_temp.dropna(axis=1, how='all') #dropping columns with 'all' missing values
africa_aez_temp = africa_aez_temp.drop(AEZ_df_as_float.index[4324]) #dropping rows with 'all' missing values
africa_aez_temp = africa_aez_temp.fillna(-9999) #dropping columns with 'all' missing values
africa_aez_no_null=africa_aez_temp.astype("int64")

africa_aez_temp = None # overwrite temp to free up memory
#AEZ_df_as_float=None


In [ ]:
africa_aez_no_null.shape

In [ ]:
AEZ_array=africa_aez_no_null.to_numpy()
AEZ_array=africa_aez_no_null.astype(int)

fig,ax=plt.subplots(figsize=(19.4,18.6))
plt.imshow(AEZ_array, aspect="auto") 
plt.show()

In [ ]:
AEZ_array.shape


In [ ]:
np.unique(AEZ_array)

In [ ]:
AEZ_raster_cleaned=os.path.join("data", "raster","AEZ_2009","afr_aez_cleaned.asc")
np.savetxt(AEZ_raster_cleaned,AEZ_array.astype(int), delimiter=" ", encoding="ANSI", fmt="%i", newline='\r\n')
#AEZ_array=None


In [ ]:
# temp_loaded=np.loadtxt(AEZ_raster_cleaned)

with open(AEZ_raster_cleaned,'r',encoding='ANSI') as f:

# write the data (but not the first 153 characters, which are metadata)
    raster_data=f.read()



In [ ]:
type(raster_data)
list(set(raster_data)) # checking all of the individual characters 

In [ ]:
with open(AEZ_raster_headers,'r',encoding='ANSI') as f:

# write the data (but not the first 153 characters, which are metadata)
    header_data=f.read()

header_data

In [ ]:
merged_file=header_data+raster_data

In [ ]:
merged_file[1:200]



In [ ]:
AEZ_RASTER_FINAL=os.path.join("data", "raster","AEZ_2009","AEZ_FINAL.asc")

with open(AEZ_RASTER_FINAL,'w',encoding='ANSI') as f:

# write the data (but not the first 153 characters, which are metadata)
    f.write(merged_file)

# close the new csv file


# Code below used to identify the NULL row which remained #

In [ ]:
# finding rows that all have the same value. This only includes rows with -9999
unique_array_values=[]
rows_with_value=[]
for rows in africa_aez_no_null.index:
    if len(list(pd.unique(africa_aez_no_null.loc[rows,:])))==1:
        if list(pd.unique(africa_aez_no_null.loc[rows,:])) in unique_array_values:
            rows_with_value.append(rows)
            continue
        else:
            unique_array_values.append(list(pd.unique(africa_aez_no_null.loc[rows,:])))
            rows_with_value.append(rows)
#unique_array_values
#rows_with_value

In [ ]:
def isNaN(num):
    return num != num

unique_array_values=[]
rows_with_nan=[]
for rows in africa_aez_no_null.index:
    unique_values_for_row=list(pd.unique(africa_aez_no_null.loc[rows,:]))
    for unique_value in unique_values_for_row:
        if isNaN(unique_value):
            rows_with_nan.append(rows)
        if unique_value in unique_array_values:
            continue
        else:
            unique_array_values.append(unique_value)

In [ ]:
#unique_array_values
rows_with_nan

In [ ]:
#africa_aez_no_null.loc[9720,:]
#see what is on either side of the nan values
either_side_4166=[]
either_side_4324=[]
either_side_4361=[]
either_side_4369=[]


row_number=4166
dataframe=africa_aez_no_null
def find_first_nan_value(row_number,dataframe):
    subset=dataframe.loc[row_number,:]
    na_location=[]
    for i in subset.index:
        if i >0 and i<9719:
            value=subset[i]
            left_subset=i-1
            right_subset=i+1
            value_left=subset[left_subset]
            value_right=subset[right_subset]
            if isNaN(value):
                if (isNaN(value_left)==False) or (isNaN(value_right)==False):
                    list_to_print=[value_left,value,value_right]
                    na_location.append(i)
    first_na_location=min(na_location)
    return first_na_location;


first_na_4166=find_first_nan_value(4166,dataframe)
first_na_4324=find_first_nan_value(4324,dataframe)
first_na_4361=find_first_nan_value(4361,dataframe)
first_na_4369=find_first_nan_value(4369,dataframe)



In [ ]:
#africa_aez_no_null.loc[4166,(first_na_4166-100):(first_na_4166+100)]
africa_aez_no_null.loc[4166,].plot()


In [ ]:
#africa_aez_no_null.loc[4324,(first_na_4324-100):(first_na_4324+100)]

africa_aez_no_null.loc[4324,].plot()




In [ ]:
#africa_aez_no_null.loc[4361,(first_na_4361-100):(first_na_4361+100)]
africa_aez_no_null.loc[4361,].plot()
#324 has almost all NA values accross the land, so it will be dropped.


In [ ]:
#africa_aez_no_null.loc[4369,(first_na_4369-100):(first_na_4369+100)]
africa_aez_no_null.loc[4369,].plot()


In [ ]:
AEZ_raster_cleaned=os.path.join("data", "raster","AEZ_2009","afr_aez_cleaned.asc")
AEZ_array.shape
#np.savetxt(AEZ_raster_cleaned,AEZ_array, delimiter=" ", newline='\n', encoding="ANSI")